In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
import optuna
# import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
#plt.style.use('fivethirtyeight')
import xgboost as xgb
import sklearn
import random
# from NecessaryModules.getData import getData

from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
# from NecessaryModules.splitData import split_data

In [6]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import mean_absolute_error

In [7]:
import pandas as pd
def getData(filename):
    chunksize = 1200000
    flag =1
    for data in pd.read_csv(filename, sep=",", chunksize=chunksize):
        while flag<2 and chunksize < 1200000:
            print(data)
        flag+=1
    print(data.head())
    print(list(data.columns.values))
    print(type(data))
    
    return data

In [8]:
filename = '../shot_listed_dataset/CoverTypeNewNormalized.csv'
dataset=getData(filename).values.tolist()
random.shuffle(dataset)
dataset=pd.DataFrame(dataset) #from dataset_list to dataset_dataframe after suffling
# print(dataset)

   Elevation    Aspect     Slope  Horizontal_Distance_To_Hydrology  \
0   0.850622  0.105556  0.150943                          0.045351   
1   0.879149  0.163889  0.188679                          0.426304   
2   0.767116  0.327778  0.320755                          0.068027   
3   0.791494  0.338889  0.226415                          0.294785   
4   0.867220  0.575000  0.584906                          0.385488   

   Vertical_Distance_To_Hydrology  Horizontal_Distance_To_Roadways  \
0                       -0.003509                         0.631060   
1                        0.189474                         0.177358   
2                        0.038596                         0.259652   
3                       -0.077193                         0.085486   
4                        0.314035                         0.074020   

   Hillshade_9am  Hillshade_Noon  Hillshade_3pm  \
0       0.866142        0.874016          0.548   
1       0.897638        0.866142          0.500   
2    

In [9]:
def split_data(data):
    # splitting the data
    
    msk = np.random.rand(len(data)) < 0.7
    yy = [54] #test title is 54 after suffleing the dataframe 
    train = data[msk]
    test = data[~msk]
    x_train = train[train.columns[:train.shape[1] - 1]]  # all columns except the last are x variables
    y_train = train[yy[0]].tolist()  # the last column as y variable
    x_test = test[test.columns[:test.shape[1] - 1]]
    y_test = test[yy[0]].tolist()  # the last column as y variable
    y_train = [0 if i == 7 else i for i in y_train]  # replace label 7 with 0
    y_test = [0 if i == 7 else i for i in y_test]  # replace label 7 with 0
    return x_train, y_train, x_test, y_test

# RFE implement

In [10]:
X=dataset.iloc[:,0:-1]
y=dataset.iloc[:,-1]

In [11]:
estimator = SVR(kernel="linear")
model = RFE(estimator, n_features_to_select=40, step=1)
rfeTrain = model.fit(X,y)
print(rfeTrain)

compressed_dataset_X= rfeTrain.transform(X)
print(type(compressed_dataset_X))
print(len(compressed_dataset_X))
print(compressed_dataset_X)


RFE(estimator=SVR(kernel='linear'), n_features_to_select=40)
<class 'numpy.ndarray'>
19229
[[0.82287344 0.         0.         ... 0.         0.         0.        ]
 [0.59258299 1.         0.         ... 0.         0.         0.        ]
 [0.89548755 0.         0.         ... 0.         0.         0.        ]
 ...
 [0.75829875 0.         0.         ... 0.         0.         0.        ]
 [0.71706432 0.         0.         ... 0.         0.         0.        ]
 [0.6501556  0.         0.         ... 0.         0.         0.        ]]


In [12]:
# from NecessaryModules.splitData import split_data
X_train, y_train, X_test, y_test = split_data(dataset)
print(len(X_train))

13516


In [13]:
compressed_dataset_X_train= rfeTrain.transform(X_train) # --transform-- the original "dataset" Dataframe into a lower-dimensional space based on the model's learned parameters
print(type(compressed_dataset_X_train))
print(len(compressed_dataset_X_train))
print(compressed_dataset_X_train)

compressed_dataset_X_test= rfeTrain.transform(X_test) # --transform-- the original "dataset" Dataframe into a lower-dimensional space based on the model's learned parameters
print(type(compressed_dataset_X_test))
print(len(compressed_dataset_X_test))
print(compressed_dataset_X_test)

<class 'numpy.ndarray'>
13516
[[0.82287344 0.         0.         ... 0.         0.         0.        ]
 [0.59258299 1.         0.         ... 0.         0.         0.        ]
 [0.58687759 1.         0.         ... 0.         0.         0.        ]
 ...
 [0.56483403 1.         0.         ... 0.         0.         0.        ]
 [0.75829875 0.         0.         ... 0.         0.         0.        ]
 [0.71706432 0.         0.         ... 0.         0.         0.        ]]
<class 'numpy.ndarray'>
5713
[[0.89548755 0.         0.         ... 0.         0.         0.        ]
 [0.57883817 1.         0.         ... 0.         0.         0.        ]
 [0.85036307 0.         0.         ... 0.         1.         0.        ]
 ...
 [0.88692946 0.         0.         ... 1.         0.         0.        ]
 [0.88070539 0.         0.         ... 0.         1.         0.        ]
 [0.6501556  0.         0.         ... 0.         0.         0.        ]]


# tune D tree for AdaBoost

In [14]:
from sklearn.tree import DecisionTreeClassifier
params_dt = {'max_depth': 12,    
             'max_features': "sqrt"}

model_dt = DecisionTreeClassifier(**params_dt)

model_dt.fit(compressed_dataset_X_train, y_train)

DecisionTreeClassifier(max_depth=12, max_features='sqrt')

In [15]:
# Finding mean absolute error
preds = model_dt.predict(compressed_dataset_X_test)
preds = preds.astype(int)
mae = np.abs(y_test-preds).mean()
mae

0.887974794328724

In [16]:
# Defining the best parameter with optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_error
K = 5 # stratified 5 fold cross validation, The folds are made by preserving the percentage of samples for each class.
skf = StratifiedKFold(n_splits = K, random_state = 42, shuffle = True)


train_tunned_dt = np.array(compressed_dataset_X)
target_train_tunned_dt = np.array(y)

# test = np.array(X)
# target_train = df_train['congestion'].values

# test_preds = np.zeros((len(test)))
oof_preds_dt = np.zeros((len(train_tunned_dt)))
abs_error_arr_dt=[]
def objective(trial):

 for train_index, val_index in (skf.split(compressed_dataset_X,y)):
    train_X, valid_X = train_tunned_dt[train_index], train_tunned_dt[val_index]
    train_Y, valid_Y = target_train_tunned_dt[train_index], target_train_tunned_dt[val_index]
    
        
        # train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.25,random_state=0)
    param = {
        'criterion':trial.suggest_categorical('criterion',["gini", "entropy", "log_loss"]), #The function to measure the quality of a split.
        'splitter':trial.suggest_categorical('splitter',["best", "random"]),
        'max_features':trial.suggest_categorical('max_features',['sqrt','log2',1,2,3,4,5,6,7,8,9,10,11]), #a categorical value can be an integer
        'random_state':trial.suggest_int('random_state',0,5),


        }
    model = DecisionTreeClassifier(**param)  
        
    model.fit(train_X,train_Y)
        
    preds = model.predict(valid_X)
    oof_preds_dt[val_index] = preds # preds gives total 3846 predicted value for valid_X


 
#  print(len(oof_preds_dt))
      
 mae = mean_absolute_error(target_train_tunned_dt, oof_preds_dt)
        
 return mae

In [17]:
# this is the main optuna for optimizing a object-->"objective"
study = optuna.create_study(direction='minimize')

[I 2024-02-19 20:40:43,228] A new study created in memory with name: no-name-1292434c-e1e5-4c64-8f58-9e583cd2378a


In [18]:

study.optimize(objective, n_trials=100) # 150 bar "objective" object ke trial kore kore dekhbe, here total 5*100=750
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2024-02-19 20:40:43,407] Trial 0 finished with value: 0.9732175360133132 and parameters: {'criterion': 'gini', 'splitter': 'random', 'max_features': 2, 'random_state': 3}. Best is trial 0 with value: 0.9732175360133132.


[I 2024-02-19 20:40:43,776] Trial 1 finished with value: 0.9767018565708045 and parameters: {'criterion': 'entropy', 'splitter': 'random', 'max_features': 8, 'random_state': 5}. Best is trial 0 with value: 0.9732175360133132.
[I 2024-02-19 20:40:44,106] Trial 2 finished with value: 0.9695772011025015 and parameters: {'criterion': 'entropy', 'splitter': 'best', 'max_features': 'sqrt', 'random_state': 1}. Best is trial 2 with value: 0.9695772011025015.
[I 2024-02-19 20:40:44,376] Trial 3 finished with value: 0.970981330282386 and parameters: {'criterion': 'log_loss', 'splitter': 'best', 'max_features': 4, 'random_state': 2}. Best is trial 2 with value: 0.9695772011025015.
[I 2024-02-19 20:40:44,596] Trial 4 finished with value: 0.9702532633002237 and parameters: {'criterion': 'log_loss', 'splitter': 'best', 'max_features': 3, 'random_state': 4}. Best is trial 2 with value: 0.9695772011025015.
[I 2024-02-19 20:40:44,806] Trial 5 finished with value: 0.9696812106713818 and parameters: {'cr

Number of finished trials: 100
Best trial: {'criterion': 'gini', 'splitter': 'random', 'max_features': 10, 'random_state': 2}


In [19]:
#evaluate the DT trial
print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials:  100
Best trial:
  Value:  0.9639086795985231
  Params: 
    criterion: gini
    splitter: random
    max_features: 10
    random_state: 2


# tune ada-boost

In [20]:
# train the model with random parameter
from sklearn.ensemble import AdaBoostClassifier
# Preparing the regressor and fitting data with some parameters
dt_params = study.best_trial.params
model_dt = DecisionTreeClassifier(**dt_params)
#model_dt = estimator
#model = predictor = model_dt.fit()

model_Ada = AdaBoostClassifier(
        estimator=model_dt,
        n_estimators=180, # maximum number of estimators at which boosting is terminated
        learning_rate=1.2,
        algorithm="SAMME", #SAMME.R=real boosting algorithm
        
     )

model_Ada.fit(compressed_dataset_X_train, y_train)

AdaBoostClassifier(algorithm='SAMME',
                   estimator=DecisionTreeClassifier(max_features=10,
                                                    random_state=2,
                                                    splitter='random'),
                   learning_rate=1.2, n_estimators=180)

In [21]:
# Finding mean absolute error
preds = model_Ada.predict(compressed_dataset_X_test)
preds = preds.astype(int)
mae = np.abs(y_test-preds).mean()
mae

0.7257132854892351

In [22]:
# Defining the best parameter with optuna

K = 5 # 5 fold cross validation
skf = StratifiedKFold(n_splits = K, random_state = 42, shuffle = True)


train_tunned_Ada= np.array(compressed_dataset_X)
target_train_tunned_Ada = np.array(y)

# test = np.array(X)
# target_train = df_train['congestion'].values

# test_preds = np.zeros((len(test)))
oof_preds_Ada = np.zeros((len(train_tunned_Ada)))
abs_error_arr_Ada=[]
def objective_Ada(trial):

 for train_index, val_index in (skf.split(compressed_dataset_X,y)):
    train_X, valid_X = train_tunned_Ada[train_index], train_tunned_Ada[val_index]
    train_Y, valid_Y = target_train_tunned_Ada[train_index], target_train_tunned_Ada[val_index]
    
        
        # train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.25,random_state=0)
    param = {
            # 'tree_method':'gpu_hist',  # this parameter means using the GPU when training our model to speedup the training process 
        'estimator':model_dt,
        'n_estimators':trial.suggest_int("n_estimators",50,500), # maximum number of estimators at which boosting is terminated
        'learning_rate':trial.suggest_float("learning_rate",1,3),
        'algorithm':trial.suggest_categorical("algorithm",["SAMME","SAMME.R"]),
        'random_state':trial.suggest_int('random_state',0,5),
        }
    model_2 = AdaBoostClassifier(**param)  
        
    model_2.fit(train_X,train_Y)
        
    preds = model_2.predict(valid_X)
    oof_preds_Ada[val_index] = preds # preds gives total 3846 predicted value for valid_X

#  abs_error=np.abs(oof_preds-target_train_tunned).mean() 
#  abs_error_arr.append(abs_error)  
#  abs_error=np.abs(oof_preds-target_train).mean()
 
#  print(len(oof_preds))
      
 mae = mean_absolute_error(target_train_tunned_Ada, oof_preds_Ada)
        
 return mae

In [23]:
# this is the main optuna for optimizing a object-->"objective"
study_Ada = optuna.create_study(direction='minimize',study_name="Ada_tunning")

[I 2024-02-19 20:41:02,370] A new study created in memory with name: Ada_tunning


In [24]:

study_Ada.optimize(objective_Ada, n_trials=100) # 150 bar "objective" object ke trial kore kore dekhbe, here total 5*100=750
print('Number of finished trials:', len(study_Ada.trials))
print('Best trial:', study_Ada.best_trial.params)

[I 2024-02-19 20:41:47,728] Trial 0 finished with value: 0.9432627801757761 and parameters: {'n_estimators': 327, 'learning_rate': 1.431387973314315, 'algorithm': 'SAMME', 'random_state': 2}. Best is trial 0 with value: 0.9432627801757761.
[I 2024-02-19 20:42:42,070] Trial 1 finished with value: 0.9496073638774767 and parameters: {'n_estimators': 399, 'learning_rate': 1.3134523208465572, 'algorithm': 'SAMME', 'random_state': 4}. Best is trial 0 with value: 0.9432627801757761.
[I 2024-02-19 20:43:27,454] Trial 2 finished with value: 0.933329866347704 and parameters: {'n_estimators': 334, 'learning_rate': 2.1588858063536778, 'algorithm': 'SAMME', 'random_state': 4}. Best is trial 2 with value: 0.933329866347704.
[I 2024-02-19 20:43:51,450] Trial 3 finished with value: 0.9552238805970149 and parameters: {'n_estimators': 166, 'learning_rate': 2.341286750718899, 'algorithm': 'SAMME', 'random_state': 5}. Best is trial 2 with value: 0.933329866347704.
[I 2024-02-19 20:44:43,261] Trial 4 finis

Number of finished trials: 100
Best trial: {'n_estimators': 466, 'learning_rate': 1.4706015912642658, 'algorithm': 'SAMME', 'random_state': 5}


# process the whole data

In [26]:
filename_2 = '../shot_listed_dataset/Cover_type_whole_genuine_dataset_normalization.csv'
dataset_2=getData(filename_2).values.tolist()
random.shuffle(dataset_2)

dataset_2=pd.DataFrame(dataset_2) #from dataset_list to dataset_dataframe after suffling
print(dataset_2)

   Elevation    Aspect     Slope  Horizontal_Distance_To_Hydrology  \
0   0.672888  0.141667  0.045455                          0.184681   
1   0.671332  0.155556  0.030303                          0.151754   
2   0.726801  0.386111  0.136364                          0.191840   
3   0.721877  0.430556  0.272727                          0.173228   
4   0.672628  0.125000  0.030303                          0.109520   

   Vertical_Distance_To_Hydrology  Horizontal_Distance_To_Roadways  \
0                        0.000000                         0.071659   
1                       -0.009983                         0.054798   
2                        0.108153                         0.446817   
3                        0.196339                         0.434172   
4                       -0.001664                         0.054939   

   Hillshade_9am  Hillshade_Noon  Hillshade_3pm  \
0       0.870079        0.913386       0.582677   
1       0.866142        0.925197       0.594488   
2    

In [27]:
X_2=dataset_2.iloc[:,0:-1]
y_2=dataset_2.iloc[:,-1]
X_train_2, y_train_2, X_test_2, y_test_2 = split_data(dataset_2)

In [28]:
# estimator = SVR(kernel="linear")
# model = RFE(estimator, n_features_to_select=30, step=1)
# rfeTrain_2 = model.fit(X_2,y_2)# --train-- pca with the "dataset" Dataframe
# print(rfeTrain_2)
compressed_dataset_X_2 = rfeTrain.transform(X_2) # --transform-- the original "dataset" Dataframe into a lower-dimensional space based on the model's learned parameters
print(type(compressed_dataset_X_2))

<class 'numpy.ndarray'>


In [29]:
compressed_dataset_X_train_2= rfeTrain.transform(X_train_2) # --transform-- the original "dataset" Dataframe into a lower-dimensional space based on the model's learned parameters
print(type(compressed_dataset_X_train_2))
print(len(compressed_dataset_X_train_2))
print(compressed_dataset_X_train_2)

compressed_dataset_X_test_2= rfeTrain.transform(X_test_2) # --transform-- the original "dataset" Dataframe into a lower-dimensional space based on the model's learned parameters
print(type(compressed_dataset_X_test_2))
print(len(compressed_dataset_X_test_2))
print(compressed_dataset_X_test_2)

<class 'numpy.ndarray'>
406295
[[0.85692068 0.         0.         ... 0.         0.         0.        ]
 [0.80248834 0.         0.         ... 0.         0.         0.        ]
 [0.66381545 0.         0.         ... 0.         0.         0.        ]
 ...
 [0.52047693 1.         1.         ... 0.         0.         0.        ]
 [0.82089165 0.         0.         ... 0.         0.         0.        ]
 [0.77164334 0.         0.         ... 0.         0.         0.        ]]
<class 'numpy.ndarray'>
174717
[[0.82711249 0.         0.         ... 0.         0.         0.        ]
 [0.66848108 0.         0.         ... 0.         0.         0.        ]
 [0.8131156  0.         0.         ... 0.         0.         0.        ]
 ...
 [0.76697771 0.         0.         ... 0.         0.         0.        ]
 [0.78512182 0.         0.         ... 0.         0.         0.        ]
 [0.57490928 1.         0.         ... 0.         0.         0.        ]]


# fit the ada-boost model

In [30]:

K = 5 # 5 fold cross validation
skf = StratifiedKFold(n_splits = K, random_state = 42, shuffle = True)

In [31]:
# Let us make predictions for each of the 5 models and find mean 
# of those predictions

from tqdm import tqdm
train_Ada = np.array(compressed_dataset_X_train_2)
target_train_Ada = np.array(y_train_2)

test_Ada = np.array(compressed_dataset_X_test_2) # or test_Ada = np.array(X)


test_preds_Ada = np.zeros((len(test_Ada)))
oof_preds_Ada = np.zeros((len(train_Ada)))

for train_index, val_index in tqdm(skf.split(compressed_dataset_X_train_2,y_train_2), total=5):
    train_X, valid_X = train_Ada[train_index], train_Ada[val_index]
    train_y, valid_y = target_train_Ada[train_index], target_train_Ada[val_index]
    ada_params = study_Ada.best_trial.params
    model_Ada_2 = AdaBoostClassifier(**ada_params,estimator=model_dt)
    model_Ada_2.fit(train_X, train_y)
    # for train accuracy
    preds = model_Ada_2.predict(valid_X)
    oof_preds_Ada[val_index] = preds
    # for test accuracy
    preds = model_Ada_2.predict(test_Ada)
    test_preds_Ada += (preds)/5
print(np.abs(oof_preds_Ada-target_train_Ada).mean())

100%|██████████| 5/5 [39:45<00:00, 477.04s/it]

0.3998350951894559


In [ ]:
#validation accuracy
valid_Acc=accuracy_score(target_train_Ada,oof_preds_Ada)
print(valid_Acc)

In [32]:
# train accuracy
train_Acc=model_Ada_2.score(compressed_dataset_X_train_2,y_train_2)
print(train_Acc)
# **** over-fitting can be terminated by taking the n_estimator manually and observe loss for each

0.7228220873995496
